In [1]:
from IPython.display import clear_output

import datetime
import time
import os


from tools import (
    play_beep,
    print_account_summary,
    print_order,
    print_all_openorders,
    get_all_openorders,
)

from IPython.display import display
from IPython.core.display import HTML

play_beep()

display(HTML("<style>.output_subarea { overflow: auto; }</style>"))

In [3]:
from ib_async import *

util.startLoop()

ib = IB()
ib.connect("127.0.0.1", 4001, clientId=123)
util.logToConsole()

In [4]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(index=range(5), columns="bidSize bidPrice askPrice askSize".split())

# NQM2024 contract
contract = Contract(conId=620730920)
ib.qualifyContracts(contract)

ticker = ib.reqMktDepth(contract)


def onTickerUpdate(ticker):
    bids = ticker.domBids
    for i in range(5):
        df.iloc[i, 0] = bids[i].size if i < len(bids) else 0
        df.iloc[i, 1] = bids[i].price if i < len(bids) else 0
    asks = ticker.domAsks
    for i in range(5):
        df.iloc[i, 2] = asks[i].price if i < len(asks) else 0
        df.iloc[i, 3] = asks[i].size if i < len(asks) else 0
    clear_output(wait=True)
    display(df)


# ticker.updateEvent += onTickerUpdate

# IB.sleep(15)

In [ ]:
ib.cancelMktDepth(contract)

In [5]:
ib.positions(account="U10394496")

[Position(account='U10394496', contract=Option(conId=637079300, symbol='HYG', lastTradeDateOrContractMonth='20240621', strike=73.0, right='P', multiplier='100', currency='USD', localSymbol='HYG   240621P00073000', tradingClass='HYG'), position=50.0, avgCost=12.44685),
 Position(account='U10394496', contract=Option(conId=682674010, symbol='AAPL', lastTradeDateOrContractMonth='20241018', strike=150.0, right='P', multiplier='100', currency='USD', localSymbol='AAPL  241018P00150000', tradingClass='AAPL'), position=2.0, avgCost=244.69685),
 Position(account='U10394496', contract=Option(conId=669333316, symbol='NVDA', lastTradeDateOrContractMonth='20240719', strike=555.0, right='P', multiplier='100', currency='USD', localSymbol='NVDA  240719P00555000', tradingClass='NVDA'), position=1.0, avgCost=493.0494),
 Position(account='U10394496', contract=Option(conId=689749183, symbol='LLY', lastTradeDateOrContractMonth='20240816', strike=620.0, right='P', multiplier='100', currency='USD', localSymbo

In [33]:
[print(trade.order.permId) for trade in trades ]

449873795
1939818441
253828840


[None, None, None]

In [34]:
# default reset values
open_trade = None
close_trade = None
# get a specific order by id

trades = get_all_openorders(ib)
open_trade = [trade for trade in trades if trade.order.permId == 1939818441]
print(open_trade)

future = [
    pos for pos in ib.positions(account="U10394496") if pos.contract.symbol == "NQ"
][0]

tick_tp = 10
ticrement = 0.25

print(future)
print_account_summary(ib=ib)

[]
Position(account='U10394496', contract=Future(conId=620730920, symbol='NQ', lastTradeDateOrContractMonth='20240621', multiplier='20', currency='USD', localSymbol='NQM4', tradingClass='NQ'), position=-2.0, avgCost=350208.07671815)
AccountType : INDIVIDUAL
Cushion : 0.775311
LookAheadNextChange : 1714570200
AccruedCash : 0.00
AvailableFunds : 164789.38
BuyingPower : 677279.44
EquityWithLoanValue : 169319.86
ExcessLiquidity : 169319.86
FullAvailableFunds : 164789.38
FullExcessLiquidity : 169319.86
FullInitMarginReq : 49835.31
FullMaintMarginReq : 45304.83
GrossPositionValue : 3764.95
InitMarginReq : 49835.31
LookAheadAvailableFunds : 179739.97
LookAheadExcessLiquidity : 182911.31
LookAheadInitMarginReq : 34884.72
LookAheadMaintMarginReq : 31713.38
MaintMarginReq : 45304.83
NetLiquidation : 218389.64
PreviousDayEquityWithLoanValue : 0.00
RegTEquity : 0.00
RegTMargin : 0.00
SMA : 623475.35
TotalCashValue : 214624.69


In [29]:
# print_all_openorders(ib=ib)
# ib.cancelOrder(
trades = get_all_openorders(ib)
print(trades[1].order)

Order(orderId=165, clientId=12, permId=1939818440, action='BUY', totalQuantity=1.0, orderType='LMT', lmtPrice=17520.5, auxPrice=0.0, tif='GTC', ocaType=3, displaySize=2147483647, trailStopPrice=17521.5, volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U10394496', clearingIntent='IB', adjustedOrderType='None', cashQty=0.0, dontUseAutoPriceForHedge=True)


In [26]:
print_all_openorders(ib=ib)

symbol	permId		status		action	filled	remaining	lmtPrice
NQ	449873795	Submitted	BUY	0.0	4.0		15705.0	
NQ	1939818440	Submitted	BUY	0.0	1.0		17520.5	
NQ	253828840	Submitted	SELL	0.0	1.0		17742.0	


In [ ]:
open_trade = None
close_trade = None

In [ ]:
## SELL TO OPEN SCALP 0.1
########################################

sell_15_10_scalp = {
    "strategy": "SELL TO OPEN SCALP 15/10",
    "contract": "NQM2024",
    "tick_increment": 0.25,
    "open_qty": 1,
    "open_type": "LIMIT",
    "open_action": "SELL",
    "open_ref": "ask",
    "open_ticks": 15,
    "close_qty": 1,
    "close_type": "LIMIT",
    "close_action": "BUY",
    "close_ref": "open_price_fill",
    "close_ticks": -10,
}

buy_10_10_scalp = {
    "strategy": "BUY TO OPEN SCALP 10/10",
    "contract": "NQM2024",
    "tick_increment": 0.25,
    "open_qty": 1,
    "open_type": "LIMIT",
    "open_action": "BUY",
    "open_ref": "bid",
    "open_ticks": -10,
    "close_qty": 1,
    "close_type": "LIMIT",
    "close_action": "SELL",
    "close_ref": "open_price_fill",
    "close_ticks": 10,
}

strategy_details = buy_10_10_scalp

print(strategy_details)

while True:
    clear_output(wait=True)  # Clear the output before printing new output

    # first order of the strategy
    if open_trade is None and close_trade is None:
        action = strategy_details["open_action"]
        qty = strategy_details["open_qty"]

        if strategy_details["open_ref"] == "bid":
            price_ref = ticker.domBids[0].price
        elif strategy_details["open_ref"] == "ask":
            price_ref = ticker.domAsks[0].price
        elif strategy_details["open_ref"] == "mid":
            price_ref = (ticker.domAsks[0].price + ticker.domBids[0].price) / 2
        elif strategy_details["open_ref"] == "last":
            raise Exception("Not implemented")

        lmtPrice = (
            price_ref
            + strategy_details["open_ticks"] * strategy_details["tick_increment"]
        )
        print(
            f"Placing open trade: {action}, {strategy_details['open_type']}, totalQuantity {qty}, lmtPrice {lmtPrice}\n"
        )
        print()

        if strategy_details["open_type"] == "LIMIT":
            open_order = LimitOrder(
                action=action,
                totalQuantity=qty,
                lmtPrice=lmtPrice,
                account="U10394496",
            )
        else:
            raise Exception("Not implemented")

        open_trade = ib.placeOrder(contract, open_order)
        open_order_ts = datetime.datetime.now()

    print("OPEN ORDER::")
    print_order(open_trade)
    print()

    if open_trade is not None:
        if open_trade.orderStatus.status == "Submitted" and close_trade is None:
            print(
                f"Waiting to get filled on order #{open_trade.order.permId} ({open_trade.orderStatus.status})\n"
            )

            if datetime.datetime.now() - open_order_ts > datetime.timedelta(minutes=1):
                print("Cancelling order due to timeout:")
                ib.cancelOrder(open_trade.order)
                print()

        if open_trade.orderStatus.status == "Filled" and close_trade is None:
            action = strategy_details["close_action"]
            qty = strategy_details["close_qty"]

            if strategy_details["close_ref"] == "open_price_fill":
                price_ref = open_trade.orderStatus.avgFillPrice
            if strategy_details["close_ref"] == "bid":
                price_ref = ticker.domBids[0].price
            elif strategy_details["close_ref"] == "ask":
                price_ref = ticker.domAsks[0].price
            elif strategy_details["close_ref"] == "mid":
                price_ref = (ticker.domAsks[0].price + ticker.domBids[0].price) / 2
            elif strategy_details["close_ref"] == "last":
                raise Exception("Not implemented")

            lmtPrice = (
                price_ref
                + strategy_details["close_ticks"] * strategy_details["tick_increment"]
            )
            print(
                f"Placing close trade: {action}, {strategy_details['close_type']}, totalQuantity {qty}, lmtPrice {lmtPrice}\n"
            )

            if strategy_details["close_type"] == "LIMIT":
                close_order = LimitOrder(
                    action=action,
                    totalQuantity=qty,
                    lmtPrice=lmtPrice,
                    account="U10394496",
                )
            else:
                raise Exception("Not implemented")

            close_trade = ib.placeOrder(contract, close_order)
            ib.sleep(1)
            play_beep()

        elif (
            open_trade.orderStatus.status == "Inactive"
            or open_trade.orderStatus.status == "Cancelled"
        ) and close_trade is None:
            print("***** order is inactive *****")
            print(open_trade.log)
            print("*****************************")
            open_trade = None

    print(f"CLOSE ORDER::")
    print_order(close_trade)
    print()

    if close_trade is not None:
        if close_trade.orderStatus.status == "Filled":
            play_beep()
            print(
                "Close trade filled @ {}\n".format(close_trade.orderStatus.avgFillPrice)
            )
            open_trade = None
            close_trade = None

            # small break before next trade
            for i in range(5):
                print(".", end="")
                time.sleep(1)

    print("ALL OPEN ORDERS::")
    print_all_openorders(ib=ib)
    print()

    # get current position of contract
    future = [pos for pos in ib.positions() if pos.contract.symbol == "NQ"][0]
    print(f"NQM2024: {ib.ticker(contract=contract).domBids[0].price}")
    print(
        f"Position: {future.contract.symbol} {future.position} @ {future.avgCost/float(contract.multiplier)}"
    )
    # acct_fields = ib.accountSummary(account="U10394496")
    print(
        f"NetLiquidation: {[v for v in ib.accountValues() if v.tag == 'NetLiquidationByCurrency' and v.currency == 'BASE' and v.account == 'U10394496'][0].value}"
    )
    print(
        f"RealizedPnL: {[v for v in ib.accountValues() if v.tag == 'RealizedPnL' and v.currency == 'BASE' and v.account == 'U10394496'][0].value}"
    )
    print(
        f"UnrealizedPnL: {[v for v in ib.accountValues() if v.tag == 'UnrealizedPnL' and v.currency == 'BASE' and v.account == 'U10394496'][0].value}"
    )
    print(
        f"OptionMarketValue: {[v for v in ib.accountValues() if v.tag == 'OptionMarketValue' and v.currency == 'BASE' and v.account == 'U10394496'][0].value}"
    )
    print(
        f"TotalCashBalance: {[v for v in ib.accountValues() if v.tag == 'TotalCashBalance' and v.currency == 'BASE' and v.account == 'U10394496'][0].value}"
    )
    print(
        f"FuturesPNL: {[v for v in ib.accountValues() if v.tag == 'FuturesPNL' and v.currency == 'BASE' and v.account == 'U10394496'][0].value}"
    )

    for i in range(3):
        print(".", end="")
        time.sleep(1)

OPEN ORDER::
symbol	permId		status		action	filled	remaining	lmtPrice
NQ	1939818176	Submitted		BUY	0.0	1.0		17513.0	

Waiting to get filled on order #1939818176 (Submitted)

CLOSE ORDER::
None

ALL OPEN ORDERS::
symbol	permId		status		action	filled	remaining	lmtPrice
NQ	449873795	Submitted	BUY	0.0	4.0		15705.0	
NQ	1939818176	Submitted	BUY	0.0	1.0		17513.0	
NQ	253828840	Submitted	SELL	0.0	1.0		17742.0	

NQM2024: 17515.0
Position: NQ -2.0 @ 17510.4038359075
NetLiquidation: 218693.6232
RealizedPnL: 1395.45
UnrealizedPnL: 555.41
OptionMarketValue: 3764.95
TotalCashBalance: 214928.6732
FuturesPNL: -485.00
..

In [ ]:
ib.accountValues()

In [ ]:
future = [pos for pos in ib.positions() if pos.contract.symbol == "NQ"][0]
future.position

In [ ]:
ib.ticker(contract=contract).domBids[0].price

In [ ]:
print(close_trade)
close_trade = ib.placeOrder(contract, close_order)
close_trade.log

In [ ]:
print_all_openorders(ib=ib)

In [ ]:
trades = ib.reqAllOpenOrders()

trades.sort(key=lambda trade: trade.order.lmtPrice)

for trade in trades:
    orderstatus = trade.orderStatus
    order = trade.order

    if trade.contract.symbol != "NQ":
        continue

    print(
        f"{trade.contract.symbol}\t{order.permId}\t{orderstatus.status}\t{order.action}\t{orderstatus.filled}\t{orderstatus.remaining}\t\t{order.lmtPrice}\t"
    )

In [ ]:
print_account_summary(ib=ib)